In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

train_data = pd.read_csv('/content/drive/MyDrive/datasets/subtask_1/es/train.tsv',sep='\t')
train_data = train_data.reset_index(drop=True)
print(train_data.head())

      id                                               text      label
0   5464  Entrada en vigor. La presente Directiva entrar...      human
1  30129  Preguntas: 1. ¿Cuáles son los principales argu...  generated
2  19553  ¿Desea algo? Póngame una caja de madera. ¿Qué ...  generated
3  13005  @victor28088 1665 Tweets no originales, que as...      human
4  16919  De pequeño Dios me dio a elegir entre tener un...      human


In [5]:
from sklearn.model_selection import train_test_split
train_data_texts = train_data['text'].to_list()
train_data_labels = train_data['label'].to_list()
train_texts, test_texts, train_labels, test_labels = train_test_split(train_data_texts, train_data_labels, test_size=0.1)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1)
print('train data size: ', len(train_texts))
print('validation data size: ', len(val_texts))
print('test data size: ', len(test_texts))


train data size:  25969
validation data size:  2886
test data size:  3207


In [6]:
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

In [7]:
import nltk
# feature augmentation using pos tags
nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
tagger = nltk.pos_tag

train_texts_pos = []
for text in train_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    train_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    train_texts_pos.append(train_pos_tags)

val_texts_pos = []
for text in val_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    val_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    val_texts_pos.append(val_pos_tags)

test_texts_pos = []
for text in test_texts:
    # Tokenize text and get POS tags
    tokens = tokenizer.tokenize(text)
    test_pos_tags = [tag for token, tag in nltk.pos_tag(tokens)]
    test_texts_pos.append(test_pos_tags)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [8]:
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-cased").to(device)
print("Model Configurations")
print()
print(model.config)

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchi

Model Configurations

BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}



In [14]:
from textblob import TextBlob

# Create a list of sentiment scores for each text
train_sentiments = [TextBlob(text).sentiment.polarity for text in train_texts]
val_sentiments = [TextBlob(text).sentiment.polarity for text in val_texts]
test_sentiments = [TextBlob(text).sentiment.polarity for text in test_texts]

# Convert the sentiment scores to a tensor
train_sentiments = torch.tensor(train_sentiments).unsqueeze(-1)
val_sentiments = torch.tensor(val_sentiments).unsqueeze(-1)
test_sentiments = torch.tensor(test_sentiments).unsqueeze(-1)

# Convert to a list of strings
train_texts_pos = [' '.join(pos_tags) for pos_tags in train_texts_pos]
val_texts_pos = [' '.join(pos_tags) for pos_tags in val_texts_pos]
test_texts_pos = [' '.join(pos_tags) for pos_tags in test_texts_pos]

# embeddings with pos
# train_encodings = tokenizer(train_texts, train_texts_pos, truncation=True, padding=True, max_length=256)
# val_encodings = tokenizer(val_texts, val_texts_pos, truncation=True, padding=True, max_length=256)
# test_encodings = tokenizer(test_texts, test_texts_pos, truncation=True, padding=True, max_length=256)

# embeddings without pos
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

# Concatenate the sentiment scores with the BERT embeddings
train_encodings.update({'sentiment': train_sentiments})
val_encodings.update({'sentiment': val_sentiments})
test_encodings.update({'sentiment': test_sentiments})

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels[idx] == 'human':
            item['labels'] = torch.tensor(0)
        else:
            item['labels'] = torch.tensor(1)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

print("Sample Data Point")
print()
print(train_dataset[0])

'''
  to rectify the warning:
  <ipython-input-9-a0a6613eed66>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

  class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).detach().clone() for key, val in self.encodings.items()}
        if self.labels[idx] == 'human':
            item['labels'] = torch.tensor(0).detach().clone()
        else:
            item['labels'] = torch.tensor(1).detach().clone()
        return item

    def __len__(self):
        return len(self.labels)
'''

Sample Data Point

{'input_ids': tensor([    4,  7600,  7690,  1036,  1785,  1008,  1967,  2542,  1553, 18550,
         1042,  2074,  1404, 16881, 30934,  1009,  1162,  6382,  1072,  4893,
         1058,  1456, 10768,  1091, 10040,  4945,  1009,  1613,  1992,  8319,
        30934,  5740,  1084,  1062, 19907,  1051,  9484,  6346,  1184,  1089,
        27221,  1042,  2179,  1108, 13795,  1216, 16736,  1009,  2779,  1017,
         6120, 30935,  1789,  1577,  1042,  2016, 20479,  1597,  1049,  1730,
         1074,  1216,  1009,  2169,  2415,  1017,  1089,  1992,  8319, 30934,
        27221,  1447,  1062, 19907,  1051,  1040,  2830, 13818,  1042, 18616,
         1013,  8142,  1150,  1888,  1008,  2438,  2701,  1009,     5,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,   

<ipython-input-14-41f7d75abbdf>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


"\n  to rectify the warning:\n  <ipython-input-9-a0a6613eed66>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).\n  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}\n\n  class CustomDataset(torch.utils.data.Dataset):\n    def __init__(self, encodings, labels):\n        self.encodings = encodings\n        self.labels = labels\n\n    def __getitem__(self, idx):\n        item = {key: torch.tensor(val[idx]).detach().clone() for key, val in self.encodings.items()}\n        if self.labels[idx] == 'human':\n            item['labels'] = torch.tensor(0).detach().clone()\n        else:\n            item['labels'] = torch.tensor(1).detach().clone()\n        return item\n\n    def __len__(self):\n        return len(self.labels)\n"

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [15]:
training_args = TrainingArguments(
    output_dir='./results',       
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    evaluation_strategy='epoch',
    save_strategy='epoch',
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-14-41f7d75abbdf>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.067200,0.667524,0.874913,0.873386,0.893108,0.874616
2,0.048300,0.271240,0.936244,0.936185,0.937646,0.936167
3,0.018300,0.449586,0.932432,0.932332,0.934720,0.932333


<ipython-input-14-41f7d75abbdf>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-14-41f7d75abbdf>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=2436, training_loss=0.044166617909732524, metrics={'train_runtime': 2876.1286, 'train_samples_per_second': 27.087, 'train_steps_per_second': 0.847, 'total_flos': 8887888366728120.0, 'train_loss': 0.044166617909732524, 'epoch': 3.0})

In [16]:
trainer.evaluate(test_dataset)
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

<ipython-input-14-41f7d75abbdf>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


<ipython-input-14-41f7d75abbdf>:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

           0       0.97      0.91      0.94      1574
           1       0.91      0.97      0.94      1633

    accuracy                           0.94      3207
   macro avg       0.94      0.94      0.94      3207
weighted avg       0.94      0.94      0.94      3207

